- [ ] Verify that allowVolumeExpansion is enabled for the storage class for the PVC. Assert if its not enabled.
- [ ] Increase the PVC Volume by the provided Value depending upon ResizeOption chosen.
- [ ] If RestartPodsAfterResize is enabled, it restarts the pods attached to the PVC. NOTE: This is not required if the kubernetes has ExpandInUsePersistentVolumes enabled.
- [ ] Verify resize by running 'df-kh' on the pod attached to the PVC.
- [ ] Send the notification on the provided slack channel.


In [4]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    assert(kubectl_command.startswith("kubectl"))

    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pvc {PVCName} -n {Namespace} --output=jsonpath={{.spec.storageClassName}}\\""
    }''')
task.configure(outputName="storageClass")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [5]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    assert(kubectl_command.startswith("kubectl"))

    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get sc {storageClass} --output=jsonpath={{.allowVolumeExpansion}}\\""
    }''')
task.configure(outputName="allowVolumeExpansion")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [6]:
if allowVolumeExpansion == "" or allowVolumeExpansion is False:
    print(f'allowVolumeExpansion disabled for storage class {storageClass}, exiting')
    assert(f'allowVolumeExpansion disabled for storage class {storageClass}')
else:
    print(f'allowVolumeExpansion enabled for storage class {storageClass}')

In [2]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Optional
from unskript.enums.aws_k8s_enums import SizingOption

from beartype import beartype
@beartype
def k8s_change_pvc_size(handle, namespace: str, name: str, resize_option: SizingOption, resize_value: float) -> str:
    # Get the current size.
    kubectl_command = f'kubectl get pvc {name} -n {namespace}  -o jsonpath={{.status.capacity.storage}}'
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    currentSize = result.stdout
    currentSizeInt = int(currentSize.rstrip("Gi"))
    if resize_option == SizingOption.Add:
        newSizeInt = currentSizeInt + resize_value
    else:
        newSizeInt = currentSizeInt * resize_value
    newSize = str(newSizeInt) + "Gi"
    print(f'Current size {currentSize}, new Size {newSize}')
    kubectl_command = f'kubectl patch pvc {name} -n {namespace} -p \'{{"spec":{{"resources":{{"requests": {{"storage": "{newSize}"}}}}}}}}\''
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None
    print(f'PVC {name} size changed to {newSize} successfully')
    return result.stdout


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(inputParamsJson='''{
    "name": "PVCName",
    "namespace": "Namespace",
    "resize_option": "SizingOption.Add if ResizeOption == \\"Add\\" else SizingOption.Mutiple",
    "resize_value": "float(Value)"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_change_pvc_size, lego_printer=unskript_default_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Optional
import enum

from beartype import beartype
@beartype
def k8s_get_pods_attached_to_pvc(handle, namespace: str, pvc: str) :
    kubectl_command = f"kubectl describe pvc {pvc} -n {namespace} | awk \'/Used By/ {{print $3}}\'"
    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None
    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "namespace": "Namespace",
    "pvc": "PVCName"
    }''')
task.configure(outputName="podName")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_get_pods_attached_to_pvc, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [ ]:
podName = podName.strip()
print(f'Pod {podName} attached to PVC {PVCName}')

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    assert(kubectl_command.startswith("kubectl"))

    result = handle.run_native_cmd(kubectl_command)
    if result.stderr:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl delete pod {podName} -n {Namespace}\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "RestartPodsAfterResize==True",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from typing import List

from kubernetes import client
from kubernetes.stream import stream
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_exec_command_on_pod(handle, namespace: str, podname: str, command: List) :
    coreApiClient = client.CoreV1Api(api_client=handle)

    try:
        resp = stream(coreApiClient.connect_get_namespaced_pod_exec,
                      podname,
                      namespace,
                      command=list(command),
                      stderr=True,
                      stdin=True,
                      stdout=True,
                      tty=False
                      )
    except:
        resp = 'An Exception occured while executing the command'

    return resp




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "[\\"df\\", \\"-kh\\"]",
    "namespace": "Namespace",
    "podname": "podName"
    }''')

(err, hdl, args) = task.validate(vars=vars())
print(args)
if err is None:
    task.output = task.execute(k8s_exec_command_on_pod, hdl, args)
if hasattr(task, 'output'):
    print(task.output)

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command(handle, kubectl_command: str) :

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return None

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pvc {PVCName} -n {Namespace}  -o jsonpath={{.status.capacity.storage}}\\""
    }''')
task.configure(outputName="newSize")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_kubectl_command, hdl, args)
if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(task.output)

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(data):
    if data != None:
        pprint.pprint(data)

@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.reponse['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.respones['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

        return f"Unable to send message on {channel}"
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "Channel",
    "message": "f\\"PVC {PVCName} successfully resized to {newSize}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)